# Data Science with Python and Dask
## Chapter 5: Cleaning and Transforming DataFrames

In [1]:
# Before beginning, set your working directory to where the data resides
import os
os.chdir('/Users/Abba/Documents/data-science-python-dask')

### Intro Section

In [2]:
# Listing 5.1
import dask
import dask.dataframe as dd
#from dask.diagnostics import ProgressBar

#import numpy as np
#import pandas as pd
#pd.set_option("string_storage", "pyarrow")
dask.config.set({"use_nullable_dtypes": True})
dask.config.set({"dataframe.convert-string": True}) # check if this is necessary, or?
dask.config.set({"dataframe.dtype_backend": "pyarrow"})

In [4]:
from dask.distributed import Client, LocalCluster, progress
cluster = LocalCluster()
client = Client(cluster)

2025-11-30 08:52:29,521 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle e2c0af88f6f4f4be1c3715fd000abb18 initialized by task ('shuffle-transfer-e2c0af88f6f4f4be1c3715fd000abb18', 50) executed on worker tcp://127.0.0.1:62384
2025-11-30 08:53:58,001 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle e2c0af88f6f4f4be1c3715fd000abb18 deactivated due to stimulus 'task-finished-1764510837.9954135'
2025-11-30 08:55:28,021 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f3a5dedf0b0fc5e5b5ef7d58359bfbb1 initialized by task ('shuffle-transfer-f3a5dedf0b0fc5e5b5ef7d58359bfbb1', 30) executed on worker tcp://127.0.0.1:62383
2025-11-30 08:56:56,869 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle f3a5dedf0b0fc5e5b5ef7d58359bfbb1 deactivated due to stimulus 'task-finished-1764511016.8678603'
2025-11-30 09:01:59,081 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 2ea79dc460501921f47d364ccf18745e initialized by task ('shuffle-transfer-2ea79dc46050

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 15.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62364,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:62385,Total threads: 4
Dashboard: http://127.0.0.1:62391/status,Memory: 3.93 GiB
Nanny: tcp://127.0.0.1:62367,


In [6]:
# start off with dtypes in dask set to pyarrow dtypes. Smaller strings and consistent with pyarrow.to_parquet()

dtypes = {
 'Date First Observed': 'string',
 'Days Parking In Effect    ': 'string',
 'Double Parking Violation': 'string',
 'Feet From Curb': 'float32',
 'From Hours In Effect': 'string',
 'House Number': 'string',
 'Hydrant Violation': 'string',
 'Intersecting Street': 'string',
 'Issue Date': 'string',
 'Issuer Code': 'float32',
 'Issuer Command': 'string',
 'Issuer Precinct': 'float32',
 'Issuer Squad': 'string',
 'Issuing Agency': 'string',
 'Law Section': 'float32',
 'Meter Number': 'string',
 'No Standing or Stopping Violation': 'string',
 'Plate ID': 'string',
 'Plate Type': 'string',
 'Registration State': 'string',
 'Street Code1': 'uint32',
 'Street Code2': 'uint32',
 'Street Code3': 'uint32',
 'Street Name': 'string',
 'Sub Division': 'string',
 'Summons Number': 'uint32',
 'Time First Observed': 'string',
 'To Hours In Effect': 'string',
 'Unregistered Vehicle?': 'string',
 'Vehicle Body Type': 'string',
 'Vehicle Color': 'string',
 'Vehicle Expiration Date': 'string',
 'Vehicle Make': 'string',
 'Vehicle Year': 'float32',
 'Violation Code': 'uint16',
 'Violation County': 'string',
 'Violation Description': 'string',
 'Violation In Front Of Or Opposite': 'string',
 'Violation Legal Code': 'string',
 'Violation Location': 'string',
 'Violation Post Code': 'string',
 'Violation Precinct': 'float32',
 'Violation Time': 'string'
}

nyc_data_raw = dd.read_csv('nyc-parking-tickets/*.csv', dtype=dtypes, usecols=dtypes.keys())
# for a smaller data set for testing:
#nyc_data_raw = dd.read_csv('nyc-parking-tickets/*.csv', dtype=dtypes, usecols=dtypes.keys())

# TESTING ONLY! write a set of csv files for using smaller test files
with ProgressBar():
    nyc_data_raw.to_csv('nyc-parking-tickets/dask-csv/*.csv')

# delete most of the files and then
# read smaller files
nyc_data_raw = dd.read_csv('nyc-parking-subset-testing/*.csv', dtype=dtypes, usecols=dtypes.keys())

### Section 5.1.1

In [7]:
# Listing 5.2
display(nyc_data_raw['Plate ID'].head())

0    GBB9093
1    62416MB
2    78755JZ
3    63009MA
4    91648MC
Name: Plate ID, dtype: string

In [8]:
# Listing 5.3
display(nyc_data_raw[['Plate ID', 'Registration State']].head())

,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


In [9]:
# Listing 5.4
columns_to_select = ['Plate ID', 'Registration State']

display(nyc_data_raw[columns_to_select].head())

,Plate ID,Registration State
0,GBB9093,NY
1,62416MB,NY
2,78755JZ,NY
3,63009MA,NY
4,91648MC,NY


### Section 5.1.2

In [10]:
# Listing 5.5
display(nyc_data_raw.drop('Violation Code', axis=1).head())

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,GY,0,2013.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,WH,0,2012.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,<NA>,0,0.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,WH,0,2010.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,BR,0,2012.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>


In [11]:
# Listing 5.6
violationColumnNames = list(filter(lambda columnName: 'Violation' in columnName, nyc_data_raw.columns))

display(nyc_data_raw.drop(violationColumnNames, axis=1).head())

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,...,Law Section,Sub Division,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb
0,1283294138,GBB9093,NY,PAS,08/04/2013,SUBN,AUDI,P,37250,13610,...,408.0,F1,BBBBBBB,ALL,ALL,GY,0,2013.0,-,0.0
1,1283294151,62416MB,NY,COM,08/04/2013,VAN,FORD,P,37290,40404,...,408.0,C,BBBBBBB,ALL,ALL,WH,0,2012.0,-,0.0
2,1283294163,78755JZ,NY,COM,08/05/2013,P-U,CHEVR,P,37030,31190,...,408.0,F7,BBBBBBB,ALL,ALL,<NA>,0,0.0,-,0.0
3,1283294175,63009MA,NY,COM,08/05/2013,VAN,FORD,P,37270,11710,...,408.0,F1,BBBBBBB,ALL,ALL,WH,0,2010.0,-,0.0
4,1283294187,91648MC,NY,COM,08/08/2013,TRLR,GMC,P,37240,12010,...,408.0,E1,BBBBBBB,ALL,ALL,BR,0,2012.0,-,0.0


### Section 5.1.3

In [12]:
# Listing 5.7
nyc_data_renamed = nyc_data_raw.rename(columns={'Plate ID':'License Plate'})
nyc_data_renamed

,Summons Number,License Plate,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
npartitions=138,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,uint32,string,string,string,string,uint16,string,string,string,uint32,uint32,uint32,string,string,float32,float32,float32,string,string,string,string,string,string,string,string,string,string,float32,string,string,string,string,string,string,string,float32,string,float32,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Section 5.1.4

In [13]:
# Listing 5.8
display(nyc_data_raw.loc[56].head(1))

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
56,1293090530,GES3519,NY,PAS,07/07/2013,40,SDN,HONDA,F,70630,...,BLACK,0,1997.0,-,0.0,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
# Listing 5.9
with ProgressBar():
    display(nyc_data_raw.loc[100:200].head(100))

In [ ]:
# Listing 5.10
with ProgressBar():
    some_rows = nyc_data_raw.loc[100:200].head(100)
some_rows.drop(range(100, 200, 2))

### Section 5.2.1

In [14]:
# Listing 5.11
missing_values = nyc_data_raw.isnull().sum()
percent_missing = ((missing_values / nyc_data_raw.index.size) * 100).persist()
progress(percent_missing)
percent_missing.compute()

Summons Number                        0.000000
Plate ID                              0.020867
Registration State                    0.000000
Plate Type                            0.000000
Issue Date                            0.000000
Violation Code                        0.000000
Vehicle Body Type                     0.564922
Vehicle Make                          0.650526
Issuing Agency                        0.000000
Street Code1                          0.000000
Street Code2                          0.000000
Street Code3                          0.000000
Vehicle Expiration Date               0.000002
Violation Location                   15.142846
Violation Precinct                    0.000002
Issuer Precinct                       0.000002
Issuer Code                           0.000002
Issuer Command                       15.018851
Issuer Squad                         15.022566
Violation Time                        0.019207
Time First Observed                  90.040886
Violation Cou

### Section 5.2.2

In [15]:
# Listing 5.12
columns_to_drop = list(percent_missing[percent_missing >= 50].index)
nyc_data_clean_stage1 = nyc_data_raw.drop(columns_to_drop, axis=1)

In [16]:
print(columns_to_drop)

['Time First Observed', 'Intersecting Street', 'Violation Legal Code', 'Unregistered Vehicle?', 'Meter Number', 'No Standing or Stopping Violation', 'Hydrant Violation', 'Double Parking Violation']


### Section 5.2.3

In [19]:
# Listing 5.13
count_of_vehicle_colors = nyc_data_clean_stage1['Vehicle Color'].value_counts().compute()
most_common_color = count_of_vehicle_colors.sort_values(ascending=False).index[0]

# Fill missing vehicle color with the most common color
nyc_data_clean_stage2 = nyc_data_clean_stage1.fillna({'Vehicle Color': most_common_color})

### Section 5.2.4

In [20]:
# Listing 5.14
# from the bug report, less than 9 rows can work; issue maybe was resolved by 2022.
# Updated to compensate for bug identified in https://github.com/dask/dask/issues/5854

# Old code:
rows_to_drop = list(percent_missing[(percent_missing > 0) & (percent_missing < 5)].index)
nyc_data_clean_stage3 = nyc_data_clean_stage2.dropna(subset=rows_to_drop)

# New code splits the rows to drop into two separate lists and chains the dropna methods to drop all the columns we want
#rows_to_drop1 =['Plate ID', 'Vehicle Body Type', 'Vehicle Make', 'Vehicle Expiration Date', 'Violation Precinct', 'Issuer Precinct', 'Issuer Code', 'Violation Time', 'Street Name']
#rows_to_drop2 =['Date First Observed', 'Law Section', 'Sub Division', 'Vehicle Color', 'Vehicle Year', 'Feet From Curb']
#nyc_data_clean_stage3 = nyc_data_clean_stage2.dropna(subset=rows_to_drop1).dropna(subset=rows_to_drop2)

### Section 5.2.5

In [21]:
# Listing 5.15
remaining_columns_to_clean = list(percent_missing[(percent_missing >= 5) & (percent_missing < 50)].index)
nyc_data_raw.dtypes[remaining_columns_to_clean]

Violation Location                   string[pyarrow]
Issuer Command                       string[pyarrow]
Issuer Squad                         string[pyarrow]
Violation County                     string[pyarrow]
Violation In Front Of Or Opposite    string[pyarrow]
House Number                         string[pyarrow]
Days Parking In Effect               string[pyarrow]
From Hours In Effect                 string[pyarrow]
To Hours In Effect                   string[pyarrow]
Violation Post Code                  string[pyarrow]
Violation Description                string[pyarrow]
dtype: object

In [22]:
# Listing 5.16
unknown_default_dict = dict(map(lambda columnName: (columnName, 'Unknown'), remaining_columns_to_clean))

In [23]:
# Listing 5.17
nyc_data_clean_stage4 = nyc_data_clean_stage3.fillna(unknown_default_dict)

In [24]:
# Listing 5.18
print(nyc_data_clean_stage4.isnull().sum().compute())
nyc_data_clean_stage4.persist()
nyc_data_clean_stage4

Summons Number                       0
Plate ID                             0
Registration State                   0
Plate Type                           0
Issue Date                           0
Violation Code                       0
Vehicle Body Type                    0
Vehicle Make                         0
Issuing Agency                       0
Street Code1                         0
Street Code2                         0
Street Code3                         0
Vehicle Expiration Date              0
Violation Location                   0
Violation Precinct                   0
Issuer Precinct                      0
Issuer Code                          0
Issuer Command                       0
Issuer Squad                         0
Violation Time                       0
Violation County                     0
Violation In Front Of Or Opposite    0
House Number                         0
Street Name                          0
Date First Observed                  0
Law Section              

VBox()

2025-11-30 09:00:21,411 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001A56CA306E0>>, <Task finished name='Task-492933' coro=<MultiProgressBar.listen() done, defined at C:\Users\Abba\miniforge3\envs\dask\Lib\site-packages\distributed\diagnostics\progressbar.py:281> exception=AttributeError("'MultiProgressWidget' object has no attribute 'elapsed_time'")>)
Traceback (most recent call last):
  File "C:\Users\Abba\miniforge3\envs\dask\Lib\site-packages\tornado\ioloop.py", line 758, in _run_callback
    ret = callback()
  File "C:\Users\Abba\miniforge3\envs\dask\Lib\site-packages\tornado\ioloop.py", line 782, in _discard_future_result
    future.result()
    ~~~~~~~~~~~~~^^
  File "C:\Users\Abba\miniforge3\envs\dask\Lib\site-packages\distributed\diagnostics\progressbar.py", line 329, in listen
    self._draw_stop(**response)
    ~~~~~~~~~~~~~~~^^^^^^^^^^^^
  Fil

nyc_data_clean_stage4.dtypes['Issue Date']

nyc_data_clean_stage4.visualize()

### Section 5.3

In [25]:
# Listing 5.19
license_plate_types = nyc_data_clean_stage4['Plate Type'].value_counts().compute()
license_plate_types

Task exception was never retrieved
future: <Task finished name='Task-537107' coro=<Client._gather.<locals>.wait() done, defined at C:\Users\Abba\miniforge3\envs\dask\Lib\site-packages\distributed\client.py:2388> exception=AllExit()>
Traceback (most recent call last):
  File "C:\Users\Abba\miniforge3\envs\dask\Lib\site-packages\distributed\client.py", line 2397, in wait
    raise AllExit()
distributed.client.AllExit


KeyboardInterrupt: 

In [ ]:
# Listing 5.20
condition = nyc_data_clean_stage4['Plate Type'].isin(['PAS', 'COM'])
plate_type_masked = nyc_data_clean_stage4['Plate Type'].where(condition, 'Other')
nyc_data_recode_stage1 = nyc_data_clean_stage4.drop('Plate Type', axis=1)
nyc_data_recode_stage2 = nyc_data_recode_stage1.assign(PlateType=plate_type_masked)
nyc_data_recode_stage3 = nyc_data_recode_stage2.rename(columns={'PlateType':'Plate Type'})

In [ ]:
# Listing 5.21
with ProgressBar():
    display(nyc_data_recode_stage3['Plate Type'].value_counts().compute())

In [ ]:
# Listing 5.22
single_color = list(count_of_vehicle_colors[count_of_vehicle_colors == 1].index)
condition = nyc_data_clean_stage4['Vehicle Color'].isin(single_color)
vehicle_color_masked = nyc_data_clean_stage4['Vehicle Color'].mask(condition, 'Other')
nyc_data_recode_stage4 = nyc_data_recode_stage3.drop('Vehicle Color', axis=1)
nyc_data_recode_stage5 = nyc_data_recode_stage4.assign(VehicleColor=vehicle_color_masked)
nyc_data_recode_stage6 = nyc_data_recode_stage5.rename(columns={'VehicleColor':'Vehicle Color'})

### Section 5.4

In [ ]:
# Listing 5.23
from datetime import datetime
issue_date_parsed = nyc_data_recode_stage6['Issue Date'].apply(lambda x: datetime.strptime(x, "%m/%d/%Y"), meta='string[pyarrow]')
nyc_data_derived_stage1 = nyc_data_recode_stage6.drop('Issue Date', axis=1)
nyc_data_derived_stage2 = nyc_data_derived_stage1.assign(IssueDate=issue_date_parsed)
nyc_data_derived_stage3 = nyc_data_derived_stage2.rename(columns={'IssueDate':'Issue Date'})

In [ ]:
# Listing 5.24
with ProgressBar():
    display(nyc_data_derived_stage3['Issue Date'].head())

nyc_data_derived_stage3['Issue Date'].head().dtype

nyc_data_derived_stage3["Issue Date"] = nyc_data_derived_stage3["Issue Date"].astype("date64[pyarrow]")

nyc_data_derived_stage3['Issue Date'].head().dtype

In [ ]:
# Listing 5.25
issue_date_month_year = nyc_data_derived_stage3['Issue Date'].apply(lambda dt: dt.strftime("%Y%m"), meta='string')
nyc_data_derived_stage4 = nyc_data_derived_stage3.assign(IssueMonthYear=issue_date_month_year)
nyc_data_derived_stage5 = nyc_data_derived_stage4.rename(columns={'IssueMonthYear':'Citation Issued Month Year'})


In [ ]:
# Listing 5.26
with ProgressBar():
    display(nyc_data_derived_stage5['Citation Issued Month Year'].head())

### Section 5.5.1

In [ ]:
# Listing 5.27
months = ['201310','201410','201510','201610','201710']
condition = nyc_data_derived_stage5['Citation Issued Month Year'].isin(months)
october_citations = nyc_data_derived_stage5[condition]

with ProgressBar():
    display(october_citations.head())

In [ ]:
# Listing 5.28
bound_date = '2016-4-25'
condition = nyc_data_derived_stage5['Issue Date'] > bound_date
citations_after_bound = nyc_data_derived_stage5[condition]

with ProgressBar():
    display(citations_after_bound.head())

### Section 5.5.1

In [ ]:
# Listing 5.29
with ProgressBar():
    condition = (nyc_data_derived_stage5['Issue Date'] > '2014-01-01') & (nyc_data_derived_stage5['Issue Date'] <= '2017-12-31')
    nyc_data_filtered = nyc_data_derived_stage5[condition]
    nyc_data_new_index = nyc_data_filtered.set_index('Citation Issued Month Year')

In [ ]:
#Did set_index work?
display(nyc_data_new_index.head(50))

nyc_data_new_index2 = nyc_data_filtered.set_index('Citation Issued Month Year')
nyc_data_new_index2.head()

nyc_data_new_index2.index

nyc_data_new_index2.columns

nyc_data_new_index = dd.read_parquet('nyc_data_date_index_not_repart')
display(nyc_data_new_index.columns)

In [ ]:
with ProgressBar():
    new_index_columns = list(nyc_data_new_index.columns)

In [ ]:
dtype_tuples = [(x, 'string') for x in new_index_columns]
schema1 = dict(dtype_tuples)
schema1 # This provides the columns, but sets everything to 'string', so manual corrections will be needed.

In [ ]:
import pyarrow as pa
with ProgressBar():
    first_partition = nyc_data_new_index.get_partition(0).compute()
    table = pa.Table.from_pandas(first_partition, preserve_index=False) #will this work with preserve index?
schema2 = table.schema

In [ ]:
schema2


nyc_data_new_index.divisions

In [ ]:
# Listing 5.30
years = ['2014', '2015', '2016', '2017']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
divisions = [year + month for year in years for month in months]
name_function = lambda x: f"data-{x}.parquet"

with ProgressBar():
    #nyc_data_new_index.to_parquet('nyc_data_date_index', engine='pyarrow', compression='snappy', schema=schema2)
    nyc_data_new_index.repartition(divisions=divisions).to_parquet('nyc_data_date_index', engine='pyarrow', compression='snappy')#, schema=schema2, name_function=name_function)#,
                            #coerce_timestamps="us",
                            #allow_truncated_timestamps=True)
    
nyc_data_new_index = dd.read_parquet('nyc_data_date_index')

In [ ]:
with ProgressBar():
    display(nyc_data_new_index['Issue Date'].head())

### Section 5.6.1

In [ ]:
# Listing 5.31
import pandas as pd
nyc_temps = pd.read_csv('nyc-temp-data.csv')

# Filtered out only the relevant months from the temperature data to accelerate the join
nyc_temps_filtered = nyc_temps[nyc_temps.monthYear.isin(divisions)]

nyc_temps_indexed = nyc_temps_filtered.set_index(nyc_temps_filtered.monthYear.astype(str))
nyc_data_with_temps = nyc_data_new_index.join(nyc_temps_indexed, how='inner')

with ProgressBar():
    display(nyc_data_with_temps.head(15))

### Section 5.6.2

In [ ]:
# Listing 5.32
fy16 = dd.read_csv('nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2016.csv', dtype=dtypes, usecols=dtypes.keys())
fy17 = dd.read_csv('nyc-parking-tickets/Parking_Violations_Issued_-_Fiscal_Year_2017.csv', dtype=dtypes, usecols=dtypes.keys())

fy1617 = fy16.append(fy17)

with ProgressBar():
    print(fy16['Summons Number'].count().compute())

with ProgressBar():
    print(fy17['Summons Number'].count().compute())

with ProgressBar():
    print(fy1617['Summons Number'].count().compute())


### Section 5.7.1

In [ ]:
# Listing 5.33
with ProgressBar():
    if not os.path.exists('nyc-final-csv'):
        os.makedirs('nyc-final-csv') 
    nyc_data_with_temps.repartition(npartitions=1).to_csv('nyc-final-csv/part*.csv')

In [ ]:
# Listing 5.33
with ProgressBar():
    if not os.path.exists('nyc-final-csv-compressed'):
        os.makedirs('nyc-final-csv-compressed')
    nyc_data_with_temps.to_csv(
        filename='nyc-final-csv-compressed/*', 
        compression='gzip', 
        sep='|', 
        na_rep='NULL', 
        header=False, 
        index=False)

### Listing 5.7.2

In [ ]:
# Listing 5.35
# Added reset_index as later versions of Dask raise an error stating the index column can't be found
with ProgressBar():
    nyc_data_with_temps.reset_index(drop=True).to_parquet('nyc_final', compression='snappy')